In [4]:
first 7 cut raw:read0`test.txt

"Monkey 0:"
"  Starting items: 79, 98"
"  Operation: new = old * 19"
"  Test: divisible by 23"
"    If true: throw to monkey 2"
"    If false: throw to monkey 3"
""


In [18]:
system "grep 'Starting items' input.txt"

"  Starting items: 76, 88, 96, 97, 58, 61, 67"
"  Starting items: 93, 71, 79, 83, 69, 70, 94, 98"
"  Starting items: 50, 74, 67, 92, 61, 76"
"  Starting items: 76, 92"
"  Starting items: 74, 94, 55, 87, 62"
"  Starting items: 59, 62, 53, 62"
"  Starting items: 62"
"  Starting items: 85, 54, 53"


In [14]:
/ parse items
get 18_"  Starting items: 79, 98"

79 98


In [19]:
system "grep 'Operation' input.txt"

"  Operation: new = old * 19"
"  Operation: new = old + 8"
"  Operation: new = old * 13"
"  Operation: new = old + 6"
"  Operation: new = old + 5"
"  Operation: new = old * old"
"  Operation: new = old + 2"
"  Operation: new = old + 3"


In [125]:
/ parse operations
get "{[old]div[;3]",(18_"  Operation: new = old * old"),"}"

{[old]div[;3] old * old}


In [115]:
system "grep 'Test: ' input.txt"

"  Test: divisible by 3"
"  Test: divisible by 11"
"  Test: divisible by 19"
"  Test: divisible by 5"
"  Test: divisible by 2"
"  Test: divisible by 7"
"  Test: divisible by 17"
"  Test: divisible by 13"


In [116]:
system "grep -n 'If true: ' input.txt"

"5:    If true: throw to monkey 2"
"12:    If true: throw to monkey 5"
"19:    If true: throw to monkey 3"
"26:    If true: throw to monkey 1"
"33:    If true: throw to monkey 2"
"40:    If true: throw to monkey 4"
"47:    If true: throw to monkey 5"
"54:    If true: throw to monkey 4"


In [117]:
system "grep -n 'If false: ' input.txt"
/ always throw, false always after true

"6:    If false: throw to monkey 3"
"13:    If false: throw to monkey 6"
"20:    If false: throw to monkey 1"
"27:    If false: throw to monkey 6"
"34:    If false: throw to monkey 0"
"41:    If false: throw to monkey 7"
"48:    If false: throw to monkey 7"
"55:    If false: throw to monkey 0"


In [118]:
/ parse tests
l:("  Test: divisible by 23";
 "    If true: throw to monkey 2";
 "    If false: throw to monkey 3")
get "{?[0=x mod ",(21_l 0),";",(29_l 1),";",(30_l 2),"]}"

{?[0=x mod 23;2;3]}


In [150]:
fi:{
        items:get 18_x 1;
        op:get "{[old]div[;3]",(18_x 2),"}";
        tests:get "{?[0=x mod ",(21_x 3),";",(29_x 4),";",(30_x 5),"]}";
    (items;op;tests)
 }
show input:fi each 7 cut raw

79 98       {[old]div[;3] old * 19}  {?[0=x mod 23;2;3]}
54 65 75 74 {[old]div[;3] old + 6}   {?[0=x mod 19;2;0]}
79 60 97    {[old]div[;3] old * old} {?[0=x mod 13;1;3]}
74          {[old]div[;3] old + 3}   {?[0=x mod 17;0;1]}


In [157]:
items:input[;0]
ops:input[;1]
tests:input[;2]

/ apply ops, tests and move items with applyAt(@)
show moveTo:tests[0] 0N!newItems:ops[0] items 0
@[;;{x,y};]/[items;moveTo;newItems]

500 620
3 3


79 98
54 65 75 74
79 60 97
74 500 620


In [158]:
turn:{[i;o;t;n]
    /show (n;i);
    moveTo:t[n] newItems:o[n] i n;
    i[n]:`long$(); / remove
    @[;;{x,y};]/[i;moveTo;newItems]
 };

turn[items;ops;tests;0]

`long$()
54 65 75 74
79 60 97
74 500 620


In [159]:
/ round
turn[;ops;tests]/[items;til count items]

20 23 27 26
2080 25 167 207 401 1046
`long$()
`long$()


In [160]:
/ 20 rounds
turn[;ops;tests]/[items;(20*c)#til c:count items]

10 12 14 26 34
245 93 53 199 115
`long$()
`long$()


In [166]:
/ add counter
turn:{[ic;o;t;n]
    i:ic 0;c:ic 1;
    moveTo:t[n] newItems:o[n] i n;
    c[n]+:count moveTo;
    i[n]:`long$(); / remove
    (@[;;{x,y};]/[i;moveTo;newItems];c)
 };

show r:turn[;ops;tests]/[(items;c#0);(20*c)#til c:count items]
prd 2#desc r 1

10 12 14 26 34 245 93 53 199 115 `long$() `long$()
101            95                7        105     


10605


In [167]:
/ part 1
input:fi each 7 cut read0`input.txt
items:input[;0]
ops:input[;1]
tests:input[;2]
r:turn[;ops;tests]/[(items;c#0);(20*c)#til c:count items]
prd 2#desc r 1

182293


In [213]:
/ not dividing by 3, instead module LCD of all test divisor to avoid int overflow
fi2:{
    items:get 18_x 1;
    op:get "{[old;lcd] mod[;lcd]",(18_x 2),"}";
    tests:get "{?[0=x mod ",(divisor:21_x 3),";",(29_x 4),";",(30_x 5),"]}";
    (items;op;tests;"J"$divisor)
 };

turn2:{[ic;o;t;lcd;n]
    i:ic 0;c:ic 1;
    moveTo:t[n] newItems:o[n][;lcd] i n;
    c[n]+:count moveTo;
    i[n]:`long$(); / remove
    (@[;;{x,y};]/[i;moveTo;newItems];c)
 };

input:fi2 each 7 cut raw
items:input[;0]
ops:input[;1]
tests:input[;2]
lcd:prd input[;3]

r:turn2[;ops;tests;lcd]/[(items;c#0);(10000*c)#til c:count items]
prd 0N!2#desc r 1

52166 52013


2713310158


In [214]:
/ part 2
input:fi2 each 7 cut read0`input.txt
items:input[;0]
ops:input[;1]
tests:input[;2]
lcd:prd input[;3]

r:turn2[;ops;tests;lcd]/[(items;c#0);(10000*c)#til c:count items]
prd 0N!2#desc r 1

234173 234155


54832778815
